В этом примере мы покажем, как с помощью API Яндекс.Геокодера можно преобразовать адреса в геокоординаты, с которым можно работать в [Datalens](https://datalens.yandex.ru)

## Установка библиотек

 * requests для API Геокодера
 * clickhouse-driver для Clickhouse

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install clickhouse-driver

Note: you may need to restart the kernel to use updated packages.


## Получение ключа для API Геокодера

В переменную API_KEY нужно записать свой ключ для API Геокодера.
Получить ключ можно по [ссылке](https://yandex.ru/dev/maps/geocoder/)

In [3]:
API_KEY = None

## Работа с API Геокодера

Напишем небольшой класс для работы с Геокодером

In [4]:
import requests
from dataclasses import dataclass

@dataclass
class YandexGeocoder:
    api_key: str
    geocoder_url: str = 'https://geocode-maps.yandex.ru/1.x'

    def adress_to_geopoint(self, address: str) -> str:
        """
        Преобразование адреса в геокоординаты в формате Datalens
        """
        response = requests.get(self.geocoder_url, params={
            'apikey': self.api_key,
            'geocode': address,
            'format': 'json',
        })
        response.raise_for_status()

        result = response.json()['response']['GeoObjectCollection']['featureMember']
        if not result:
            return None

        lon, lat = result[0]['GeoObject']['Point']['pos'].split(' ')
        return self._to_datalens_format(lon, lat)
    
    def _to_datalens_format(self, lon, lat):
        return f'[{lon},{lat}]'

## Получение данных

Будем работать с данными из демонстрационного Clickhouse [Datalens](https://datalens.yandex.ru).

На первом шаге подготовим клиент Clickhouse

In [5]:
from clickhouse_driver import Client as CHClient

ch_client = CHClient(
    'rc1a-ckg8nrosr2lim5iz.mdb.yandexcloud.net',
    user='samples_ro',
    password='MsgfcjEhJk',
    database='samples',
    port=9440,
    secure=True,
)

Затем выгрузим данные из таблицы в переменную ch_data

In [6]:
ch_data = ch_client.execute('SELECT ShopName, ShopAddress FROM MS_Shops')
ch_data

[('Тау', 'город Москва, Флотская улица, дом 25'),
 ('Альфа', 'город Москва, Гвардейская улица, дом 15, корпус 2'),
 ('Бета', 'город Москва, Долгоруковская улица, дом 23, строение 1'),
 ('Гамма', 'город Москва, Лазоревый проезд, дом 15, строение 4'),
 ('Дельта', 'город Москва, микрорайон Северное Чертаново, дом 5, корпус Г'),
 ('Эпсилон', 'город Москва, Ореховый проезд, дом 17, корпус 2'),
 ('Дзета', 'город Москва, Рязанский проспект, дом 2, строение 24'),
 ('Эта', 'город Москва, улица Вилиса Лациса, дом 23, корпус 2'),
 ('Йота', 'город Москва, улица Газопровод, дом 9А'),
 ('Сигма', 'город Москва, улица Демьяна Бедного, дом 8'),
 ('Омега', 'город Москва, улица Новаторов, дом 4, корпус 8'),
 ('Каппа', 'город Москва, улица Стромынка, дом 6')]

## Геокодирование

Преобразуем адреса магазинов из колонки ShopAddress в геокоординаты

In [7]:
geocoder = YandexGeocoder(api_key=API_KEY)

In [8]:
encoded_data = [
    (name, geocoder.adress_to_geopoint(adress))
    for name, adress in ch_data
]
encoded_data

[('Тау', '[37.513134,55.861615]'),
 ('Альфа', '[37.422063,55.724763]'),
 ('Бета', '[37.602939,55.775788]'),
 ('Гамма', '[37.631308,55.853819]'),
 ('Дельта', '[37.591054,55.638053]'),
 ('Эпсилон', '[37.734084,55.623697]'),
 ('Дзета', '[37.735854,55.730694]'),
 ('Эта', '[37.422297,55.862015]'),
 ('Йота', '[37.606855,55.59103]'),
 ('Сигма', '[37.488053,55.772223]'),
 ('Омега', '[37.516934,55.665121]'),
 ('Каппа', '[37.687327,55.791243]')]

## Сохранение в Clickhouse



Результат можно записать в Clickhouse. Для этого можно раскомментировать строки ниже и заполнить переменные USER_CH_HOST, USER_CH_PORT, USER_CH_USER и USER_CH_PASSWORD данными своей инсталяции базы.

Из полученной таблицы можно создать подключение в [Datalens](https://datalens.yandex-team.ru/docs/operations/connection/create-clickhouse)

При создании подключения для поля с координатами нужно выбрать тип [GEOPOINT](https://cloud.yandex.ru/docs/datalens/function-ref/GEOPOINT)

In [9]:
# USER_CH_HOST = 'localhost'
# USER_CH_PORT = 9000
# USER_CH_USER = None
# USER_CH_PASSWORD = None

# ch_client = CHClient(
#     USER_CH_HOST,
#     user=USER_CH_USER,
#     password=USER_CH_PASSWORD,
#     port=USER_CH_PORT,
#     secure=True,
# )
# ch_client.execute('DROP TABLE MS_Shops')
# ch_client.execute(
#     'CREATE TABLE IF NOT EXISTS MS_Shops (name String, adress String) ENGINE = MergeTree() ORDER BY name',
# )
# ch_client.execute(
#     'INSERT INTO MS_Shops (name, adress) VALUES',
#     [{'name': name, 'adress': adress} for name, adress in encoded_data],
# )
# ch_client.execute('SELECT * FROM MS_Shops')

## Сохранение в формате csv

In [10]:
import csv
import sys

csv_writer = csv.writer(
    sys.stdout,
    delimiter=',',
    quotechar='"',
    quoting=csv.QUOTE_MINIMAL,
)
csv_writer.writerows(encoded_data)

Тау,"[37.513134,55.861615]"
Альфа,"[37.422063,55.724763]"
Бета,"[37.602939,55.775788]"
Гамма,"[37.631308,55.853819]"
Дельта,"[37.591054,55.638053]"
Эпсилон,"[37.734084,55.623697]"
Дзета,"[37.735854,55.730694]"
Эта,"[37.422297,55.862015]"
Йота,"[37.606855,55.59103]"
Сигма,"[37.488053,55.772223]"
Омега,"[37.516934,55.665121]"
Каппа,"[37.687327,55.791243]"


Результат можно записать в файл. Для этого можно раскомментировать строки ниже и при необходимости поменять имя файла в переменной filename.

Из полученного csv можно создать файловое подключение в [Datalens](https://datalens.yandex.ru/connections/new/file)

Поле с координатами при создании датасета будет иметь тип [GEOPOINT](https://cloud.yandex.ru/docs/datalens/function-ref/GEOPOINT)

In [11]:
# filename = 'encoded_data.csv'

# with open(filename, 'w') as f:
#     csv_writer = csv.writer(
#         f,
#         delimiter=',',
#         quotechar='"',
#     )
#     csv_writer.writerows(encoded_data)